In [1]:
import pandas as pd
import funzioni as fx
lista_ticker = pd.read_parquet("lista_ticker.parquet")
lista_ticker.sample(frac=1)
tot_ticker = 800
bilanciamento = 0.5
soglia_memoria = 14.5 #GB
X, Y = fx.set_di_tuning(lista_ticker, tot_ticker, 'dati', f'_{tot_ticker}_{bilanciamento}', soglia_memoria=soglia_memoria, bilanciamento=bilanciamento)
print(X.shape)
print(Y.shape)

Using TensorFlow backend
1 di 2000: Ticker OOMA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(374, 60, 33)
(374, 1)
(374)
5.6 MB
2 di 2000: Ticker CE
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(561, 60, 33)
(561, 1)
(935)
14.1 MB
3 di 2000: Ticker CHEF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(737, 60, 33)
(737, 1)
(1672)
25.3 MB
4 di 2000: Ticker ZIP
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(110, 60, 33)
(110, 1)
(1782)
26.9 MB
5 di 2000: Ticker PII
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(693, 60, 33)
(693, 1)
(2475)
37.4 MB
6 di 2000: Ticker CARR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(198, 60, 33)
(198, 1)
(


1 Failed download:
['MONC']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
84 di 2000: Ticker WDAY
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(550, 60, 33)
(550, 1)
(39963)
603.7 MB
85 di 2000: Ticker GPN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(638, 60, 33)
(638, 1)
(40601)
613.3 MB
86 di 2000: Ticker HTLF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(605, 60, 33)
(605, 1)
(41206)
622.5 MB
87 di 2000: Ticker STRL
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(528, 60, 33)
(528, 1)
(41734)
630.4 MB
88 di 2000: Ticker CM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(660, 60, 33)
(660, 1)
(42394)
640.4 MB
89 di 2000: Ticker MEI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a


1 Failed download:
['ELN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2010-01-01 -> 2023-12-31)')


single positional indexer is out-of-bounds
104 di 2000: Ticker BRFS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(550, 60, 33)
(550, 1)
(49159)
742.6 MB
105 di 2000: Ticker ORCL
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(682, 60, 33)
(682, 1)
(49841)
752.9 MB
106 di 2000: Ticker EC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(528, 60, 33)
(528, 1)
(50369)
760.9 MB
107 di 2000: Ticker HTZ
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(88, 60, 33)
(88, 1)
(50457)
762.2 MB
108 di 2000: Ticker PFG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(550, 60, 33)
(550, 1)
(51007)
770.5 MB
109 di 2000: Ticker UCBI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta da

...


1 Failed download:
['EPR^PRE']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
780 di 2000: Ticker BLDR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(627, 60, 33)
(627, 1)
(357786)
5.3 GB
781 di 2000: Ticker WTBA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(649, 60, 33)
(649, 1)
(358435)
5.3 GB
782 di 2000: Ticker SIX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(561, 60, 33)
(561, 1)
(358996)
5.3 GB
783 di 2000: Ticker FDP
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(484, 60, 33)
(484, 1)
(359480)
5.3 GB
784 di 2000: Ticker PDD
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(242, 60, 33)
(242, 1)
(359722)
5.3 GB
785 di 2000: Ticker NETI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati


1 Failed download:
['MOG^A']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
790 di 2000: Ticker HRI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(517, 60, 33)
(517, 1)
(361141)
5.3 GB
791 di 2000: Ticker ONB
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(583, 60, 33)
(583, 1)
(361724)
5.3 GB
792 di 2000: Ticker PAHC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(374, 60, 33)
(374, 1)
(362098)
5.3 GB
793 di 2000: Ticker REGN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(803, 60, 33)
(803, 1)
(362901)
5.4 GB
794 di 2000: Ticker ABM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati a liste X e Y
(363, 60, 33)
(363, 1)
(363264)
5.4 GB
795 di 2000: Ticker LFUS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Aggiunta dati

KeyboardInterrupt: 